In [1]:
%pip install python-dotenv --upgrade --quiet langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.2 MB/s eta 0:00:00


In [2]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)
parser = StrOutputParser()

Enter your Google API Key: ··········


In [3]:
structured_prompt = """
# Context
You are a Senior Python Developer known for writing clean, well-documented code.

# Objective
Write a Python function that reverses a string.

# Constraints
1. You MUST use recursion.
2. You MUST NOT use slicing (e.g., [::-1]) or built-in reverse functions.
3. Handle the base case explicitly.

# Style
Include a detailed docstring that covers:
- What the function does
- Parameters and their types
- Return value and type
- How the recursion works (step-by-step explanation in the docstring)
- A usage example

# Output Format
Return only the Python code block. No extra explanation outside the code.
"""

result = llm.invoke(structured_prompt)
print(result.content)

```python
def reverse_string_recursive(s: str) -> str:
    """
    Reverses a given string using recursion.

    This function takes a string and returns a new string with its characters
    in reverse order. It adheres to the constraints of not using slicing
    like `[::-1]` or built-in reverse functions, and explicitly handles
    the base case.

    Parameters:
        s (str): The input string to be reversed.

    Returns:
        str: The reversed string.

    How the recursion works:
    1.  **Base Case**:
        If the input string `s` is empty (has a length of 0) or contains
        only one character, it is considered already reversed. In this scenario,
        the function returns `s` itself, which stops the recursion.
        -   Example: `reverse_string_recursive("")` returns `""`
        -   Example: `reverse_string_recursive("a")` returns `"a"`

    2.  **Recursive Step**:
        For any string `s` with two or more characters, the function performs
        the followin

In [4]:
llm_creative = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

prompt_zero = "Give a sarcastic one-line review of 'going to the gym'."
print("--- ZERO-SHOT ---")
print(llm_creative.invoke(prompt_zero).content)

prompt_few = """
Give a sarcastic one-line review of everyday activities.

Activity: Eating salad
Review: Nothing says 'I've given up on joy' quite like a bowl of sadness with dressing.

Activity: Waking up early
Review: Ah yes, voluntarily ruining your sleep to 'get things done' — a bold delusion.

Activity: Going to the gym
Review:
"""
print("\n--- FEW-SHOT ---")
print(llm_creative.invoke(prompt_few).content)

--- ZERO-SHOT ---
Oh, the sheer delight of paying good money to feel utterly exhausted and question all my life choices for an hour.

--- FEW-SHOT ---
The only place where paying to torture yourself is considered 'self-improvement'.


In [5]:
examples = [
    {"input": "The internet is down.",       "output": "We are observing connectivity latency."},
    {"input": "This code has a bug.",         "output": "The logic exhibits unintended behavior."},
    {"input": "I hate this feature.",         "output": "This feature does not align with my preferences."},
    {"input": "The meeting was a waste.",     "output": "The session yielded suboptimal value alignment."},
]

example_fmt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai",    "{output}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_fmt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Corpo-Speak Translator. Rewrite casual or negative input to sound professional and corporate."),
    few_shot_prompt,
    ("human", "{text}")
])

chain = final_prompt | llm | parser

test_inputs = [
    "This app sucks.",
    "Nobody reads the docs.",
    "We shipped broken code again."
]

for text in test_inputs:
    print(f"Input : {text}")
    print(f"Output: {chain.invoke({'text': text})}")
    print()

Input : This app sucks.
Output: The application's current iteration presents opportunities for enhancement.

Input : Nobody reads the docs.
Output: User engagement with the available documentation is currently limited.

Input : We shipped broken code again.
Output: The latest release has highlighted opportunities to reinforce our quality assurance framework.

